# 1. Deep Learning with PyTorch - A 60 Minute Blitz 
## 3. Newral Networks

URL : https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

### Contents
1. Define the network
2. Loss Function
3. Backprop
4. Update the wegiths


# 1. Define the network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3) # (input batch), output channel, 3x3 conv
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # Affine 변환 : 행렬의 내적
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [2]:
x = torch.ones(4,4)
print(x)
print(x.size())
print(x.size()[1:])

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
torch.Size([4, 4])
torch.Size([4])


In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [5]:
for i in params:
    print(i.size())

torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [6]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0660,  0.1501, -0.1336,  0.0411, -0.0075, -0.0243, -0.0142, -0.0152,
         -0.0208, -0.0433]], grad_fn=<AddmmBackward>)


In [7]:
net.zero_grad()
out.backward(torch.randn(1,10))

# 2. Loss Function

In [8]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1) # 1x10
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.2093, grad_fn=<MseLossBackward>)


In [9]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear : self.fc3
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

# 3. Backprop

In [10]:
net.zero_grad()
print(net.conv1.bias.grad)
loss.backward()
print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([-0.0019,  0.0038, -0.0023, -0.0045,  0.0247, -0.0079])


# 4. Update the wegiths

SGD(Stochastic Gradient Descent) : weight = weight - learning_rate * gradient

In [11]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [12]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()